In [ ]:
import torch
import torch.nn as nn  
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import save_image


In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = datasets.ImageFolder(root="C:/Users/Arun/pytorch/datasxts/celebA",  transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)


In [ ]:
from encoder import Encoder

# Define a simple args class to hold the parameters
class Args:
    def __init__(self, image_channels, latent_dim):
        self.image_channels = image_channels
        self.latent_dim = latent_dim

# Initialize the arguments
args = Args(image_channels=3, latent_dim=64)  # For example, using RGB images and a latent dimension of 64

# Instantiate the Encoder
encoder = Encoder(args)

# Print the model architecture (optional)
print(encoder)

# Create a random input tensor with the shape (batch_size, image_channels, height, width)
batch_size = 4
image_height = 256
image_width = 256
input_tensor = torch.randn(batch_size, args.image_channels, image_height, image_width)

# Forward pass through the encoder
output = encoder(input_tensor)

# Print the output shape
print("Output shape:", output.shape)


Encoder(
  (model): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ResBlock(
      (activation): SiLU()
      (blocks): Sequential(
        (0): GroupNorm(32, 128, eps=1e-06, affine=True)
        (1): SiLU()
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): GroupNorm(32, 128, eps=1e-06, affine=True)
        (4): SiLU()
        (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (residual): Identity()
    )
    (2): ResBlock(
      (activation): SiLU()
      (blocks): Sequential(
        (0): GroupNorm(32, 128, eps=1e-06, affine=True)
        (1): SiLU()
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): GroupNorm(32, 128, eps=1e-06, affine=True)
        (4): SiLU()
        (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (residual): Identity()
    )
    (3): DownsampleBlock(
     

In [ ]:
from decoder import Decoder


# Define a simple args class to hold the parameters
class Args:
    def __init__(self, image_channels, latent_dim):
        self.image_channels = image_channels
        self.latent_dim = latent_dim

# Initialize the arguments
args = Args(image_channels=3, latent_dim=64)  # For example, using RGB images and a latent dimension of 64

# Instantiate the Decoder
decoder = Decoder(args)

# Print the model architecture (optional)
print(decoder)

# Create a random input tensor with the shape (batch_size, latent_dim, height, width)
batch_size = 4
latent_height = 16
latent_width = 16
input_tensor = torch.randn(batch_size, args.latent_dim, latent_height, latent_width)

# Forward pass through the decoder
output = decoder(input_tensor)

# Print the output shape
print("Output shape:", output.shape)


Decoder(
  (model): Sequential(
    (0): Conv2d(64, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ResBlock(
      (activation): SiLU()
      (blocks): Sequential(
        (0): GroupNorm(32, 512, eps=1e-06, affine=True)
        (1): SiLU()
        (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): GroupNorm(32, 512, eps=1e-06, affine=True)
        (4): SiLU()
        (5): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (residual): Identity()
    )
    (2): NonLocalBlock(
      (gn): GroupNorm(32, 512, eps=1e-06, affine=True)
      (theta): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
      (phi): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
      (g): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
      (output_conv): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
    )
    (3): ResBlock(
      (activation): SiLU()
      (blocks): Sequential(
        (0): GroupNorm(32, 512, ep

In [ ]:
from encoder import Encoder
from decoder import Decoder
from vqema import VectorQuantizerEMA
class Args:
    def __init__(self, image_channels, latent_dim):
        self.image_channels = image_channels
        self.latent_dim = latent_dim

# Initialize the arguments
args = Args(image_channels=3, latent_dim=64)
class VQVAE(nn.Module):
    def __init__(self, in_channels, num_embeddings, embedding_dim, commitment_cost, decay):
        super(VQVAE, self).__init__()
        self.encoder = Encoder(args=args)
        self.vq = VectorQuantizerEMA(num_embeddings, embedding_dim, commitment_cost, decay)
        self.decoder = Decoder(args=args)

    def forward(self, x):
        z_e = self.encoder(x)
        vq_loss, z_q, _ = self.vq(z_e)
        x_recon = self.decoder(z_q)
        recon_loss = F.mse_loss(x_recon, x)
        loss = recon_loss + vq_loss
        return loss, x_recon

In [ ]:
from torchsummary import summary

in_channels = 3
num_embeddings = 512
embedding_dim = 64
commitment_cost = 0.25
decay = 0.99
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Initialize models

vqvae = VQVAE(in_channels, num_embeddings, embedding_dim, commitment_cost, decay).to(device)
en = Encoder(args=args).to(device)

summary(en, (3, 256, 256))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 256, 256]           3,584
          Identity-2        [-1, 128, 256, 256]               0
         GroupNorm-3        [-1, 128, 256, 256]             256
              SiLU-4        [-1, 128, 256, 256]               0
              SiLU-5        [-1, 128, 256, 256]               0
            Conv2d-6        [-1, 128, 256, 256]         147,584
         GroupNorm-7        [-1, 128, 256, 256]             256
              SiLU-8        [-1, 128, 256, 256]               0
              SiLU-9        [-1, 128, 256, 256]               0
           Conv2d-10        [-1, 128, 256, 256]         147,584
         ResBlock-11        [-1, 128, 256, 256]               0
         Identity-12        [-1, 128, 256, 256]               0
        GroupNorm-13        [-1, 128, 256, 256]             256
             SiLU-14        [-1, 128, 2

In [ ]:
print('Number of params in Model: {}'.format(
    sum(p.data.nelement() for p in vqvae.parameters() if p.requires_grad),
))

Number of params in Model: 62573379


In [8]:
from tqdm import tqdm
import os


# Hyperparameters
in_channels = 3
num_embeddings = 512
embedding_dim = 64
commitment_cost = 0.25
decay = 0.99
epochs = 20
learning_rate = 1e-3
batch_size = 64

model_dir = "checkpoints"
sample_dir = "samples"

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate model
vqvae = VQVAE(in_channels, num_embeddings, embedding_dim, commitment_cost, decay).to(device)

# Optimizer
optimizer = optim.Adam(vqvae.parameters(), lr=learning_rate)

# Ensure directories exist
os.makedirs(sample_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

def find_latest_checkpoint(checkpoints_dir):
    checkpoints = [f for f in os.listdir(checkpoints_dir) if f.endswith('.pth')]
    if not checkpoints:
        return None
    latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[2].split('.')[0]))
    return os.path.join(checkpoints_dir, latest_checkpoint)

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    vqvae.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    return start_epoch

# Load the latest checkpoint if available
start_epoch = 0
latest_checkpoint = find_latest_checkpoint(model_dir)
if latest_checkpoint:
    start_epoch = load_checkpoint(latest_checkpoint)
    print(f'Resuming training from epoch {start_epoch}')
else:
    print(f'Training from Scratch')

# Training loop
for epoch in range(start_epoch, epochs):
    train_loss = 0
    with tqdm(enumerate(train_loader, start=1), total=len(train_loader)) as t:
        for batch_idx, (data, _) in t:
            print(batch_idx)
            data = data.to(device)
            
            optimizer.zero_grad()
            loss, _ = vqvae(data)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            t.set_description(f'Epoch [{epoch+1}/{epochs}]')
            t.set_postfix({'Training loss': f'{train_loss/(batch_idx+1):.3f}', 'Batch loss': f'{loss:.3f}'})
            

    train_loss /= len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{epochs} Average Loss: {train_loss}')

    # Save model checkpoint
    if (epoch + 1) % 5 == 0:
        checkpoint_path = os.path.join(model_dir, f'vqvae_epoch_{epoch+1}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': vqvae.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch+1}")

    # Save some reconstructed images for visualization
    vqvae.eval()
    with torch.no_grad():
        sample = next(iter(train_loader))[0].to(device)
        _, recon_sample = vqvae(sample)
        recon_sample = recon_sample.cpu()
        save_image(recon_sample, os.path.join(sample_dir, f'epoch_{epoch+1}_reconstructions.png'))
        save_image(sample.cpu(), os.path.join(sample_dir, f'epoch_{epoch+1}_originals.png'))


Training from Scratch


  0%|          | 0/25325 [00:00<?, ?it/s]

1


Epoch [1/20]:   0%|          | 1/25325 [00:16<114:59:43, 16.35s/it, Training loss=0.607, Batch loss=1.214]

2


...

47


Epoch [1/20]:   0%|          | 47/25325 [12:20<129:44:30, 18.48s/it, Training loss=1.609, Batch loss=1.216]

48


Epoch [1/20]:   0%|          | 48/25325 [12:40<133:52:47, 19.07s/it, Training loss=1.608, Batch loss=1.531]

49
